In [ ]:
pip install langchain unstructured openai chromadb

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import DirectoryLoader, UnstructuredFileLoader
import os
# os.environ['OPENAI_API_KEY'] = '...'

In [54]:
# loader = DirectoryLoader('./data/', glob='**/*.txt')
loader = UnstructuredFileLoader('./data/labor_standard_act.txt')
docs = loader.load()
print(len(docs))

1


In [67]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
docsearch = Chroma.from_documents(texts, embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [87]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch)
# from langchain.chains import RetrievalQAWithSourcesChain
# qa = RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="map_reduce", retriever=docsearch.as_retriever())

In [88]:
qa.run("can employer prohibit employee to do the similar job after resign?")
# qa({"question": "if I worked for two years, how long should I notify the company before I resign?"}, return_only_outputs=True)

' Twenty days in advance.'